# Import libraries

In [1]:
import os
import warnings

import numpy as np
import pandas as pd

import json

from tqdm import tqdm

from itertools import combinations

In [2]:
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

In [3]:
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

In [4]:
path_to_save = './../results/03a_features'

# Loading data

In [5]:
path_to_ftir_data= './../data/processed_data/ftir_extracted_features.csv'
path_to_samples_description = './../data/parsed_data/samples_description.csv'

## FTIR-features

In [6]:
features_df = pd.read_csv(path_to_ftir_data, sep=';', index_col=0)

print(features_df.shape)
features_df.head(3)

(75, 54)


,$I_{3720}$,$I_{3533}$,$I_{3405}$,$I_{3302}$,$I_{3201}$,$I_{3132}$,$I_{3001}$,$I_{2926}$,$I_{2560}$,$I_{2464}$,$I_{2345}$,$I_{2239}$,$I_{1737}$,$I_{1704}$,$I_{1651}$,$I_{1609}$,$I_{1584}$,$I_{1557}$,$I_{1526}$,$I_{1428}$,$I_{1392}$,$I_{1237}$,$I_{1180}$,$I_{1120}$,$I_{1077}$,...,$\langle I_{3500\text{–}3600} \rangle$,$\langle I_{3400\text{–}3500} \rangle$,$\langle I_{3300\text{–}3400} \rangle$,$\langle I_{3200\text{–}3300} \rangle$,$\langle I_{3100\text{–}3200} \rangle$,$\langle I_{3000\text{–}3100} \rangle$,$\langle I_{2900\text{–}3000} \rangle$,$\langle I_{2800\text{–}2900} \rangle$,$\langle I_{2700\text{–}2800} \rangle$,$\langle I_{2600\text{–}2700} \rangle$,$\langle I_{2500\text{–}2600} \rangle$,$\langle I_{2400\text{–}2500} \rangle$,$\langle I_{2300\text{–}2400} \rangle$,$\langle I_{2200\text{–}2300} \rangle$,$\langle I_{1800\text{–}1900} \rangle$,$\langle I_{1700\text{–}1800} \rangle$,$\langle I_{1600\text{–}1700} \rangle$,$\langle I_{1500\text{–}1600} \rangle$,$\langle I_{1400\text{–}1500} \rangle$,$\langle I_{1300\text{–}1400} \rangle$,$\langle I_{1200\text{–}1300} \rangle$,$\langle I_{1100\text{–}1200} \rangle$,$\langle I_{1000\text{–}1100} \rangle$,$\langle I_{900\text{–}1000} \rangle$,$\langle I_{800\text{–}900} \rangle$
division_1_size_bulk,0.084601,0.506784,0.938564,1.000000,0.815223,0.720771,0.498252,0.422644,0.219016,0.143296,0.248711,0.118391,0.378890,0.423317,0.827933,1.0,0.761398,0.494486,0.328826,0.682168,0.819844,0.550066,0.515502,0.745328,0.830578,...,0.480218,0.812554,0.983919,0.897541,0.741429,0.569872,0.444641,0.372588,0.312491,0.253459,0.201962,0.143976,0.210565,0.122707,0.045607,0.311623,0.778964,0.505824,0.540330,0.717250,0.556714,0.630505,0.799233,0.400758,0.277182
division_1_size_5,0.038211,0.652508,1.000000,0.942571,0.814723,0.704920,0.486172,0.448983,0.201775,0.158787,0.191185,0.123498,0.371442,0.417765,0.842104,1.0,0.731738,0.455388,0.277878,0.621557,0.738359,0.508943,0.515948,0.705649,0.664996,...,0.626761,0.904449,0.981529,0.880072,0.733122,0.560306,0.455227,0.372296,0.307501,0.249682,0.196696,0.156315,0.173257,0.125642,0.024109,0.294946,0.780563,0.465631,0.487759,0.631607,0.510142,0.605438,0.563534,0.321487,0.241591
division_1_size_3,0.074497,0.598526,1.000000,0.937402,0.760651,0.637583,0.450708,0.547936,0.203027,0.137535,0.304164,0.126319,0.312673,0.408867,0.852727,1.0,0.819926,0.589477,0.391324,0.644534,0.739963,0.475526,0.618890,0.953037,0.879241,...,0.567042,0.861059,0.977271,0.853865,0.668995,0.512242,0.503021,0.405550,0.297746,0.245957,0.190836,0.133271,0.243778,0.130666,0.023757,0.252216,0.795671,0.572212,0.521961,0.605289,0.478551,0.787526,0.755641,0.403330,0.264633


## Samples description

In [7]:
samples_description = pd.read_csv(f'{path_to_samples_description}', sep=';', index_col=0)

print(samples_description.shape)
samples_description.head(3)

(75, 6)


,Row_ID,Division,Size,Fraction_hue,Fraction_grouped_hue,Class
division_1_size_bulk,1,1,bulk,$> 0$,$d > 5$,2
division_1_size_5,2,1,5,$< 5$,$2 < d \leq 5$,1
division_1_size_3,3,1,3,$< 3$,$2 < d \leq 5$,1


# Feature engineering

## Base parameters

In [8]:
features_peaks = [col for col in features_df.columns if '$\langle' not in col]
features_intervals = [col for col in features_df.columns if '$\langle' in col]

print(
    f'(*) Типа 1: {len(features_peaks)}',
    f'(*) Типа 2: {len(features_intervals)}',
    sep='\n'
)

(*) Типа 1: 27
(*) Типа 2: 27


## Function for processing

In [9]:
def generate_transformed_features(data, feature_list):
    data_transformed = pd.DataFrame(index=data.index)

    def clean(col):
        return col.strip('$')

    def wrap_log(col):
        return fr"$\log {clean(col)}$"

    def wrap_pow(col, power):
        return fr"$({clean(col)})^{{{power}}}$"

    def wrap_div(col1, col2):
        return fr"$\frac{{{clean(col1)}}}{{{clean(col2)}}}$"

    for col in feature_list:
        data_transformed[wrap_log(col)] = np.log(data[col])
        
        powers_range = [-2, -1, 2]
        
        for power in powers_range:
            data_transformed[wrap_pow(col, power)] = np.power(data[col], power)

    for f1, f2 in combinations(feature_list, 2):
        data_transformed[wrap_div(f1, f2)] = data[f1] / data[f2]

    return data_transformed

## Calculation process

In [10]:
engineered_features_peaks_df = generate_transformed_features(features_df, features_peaks)

print(engineered_features_peaks_df.shape)
engineered_features_peaks_df.head(3)

(75, 459)


,$\log I_{3720}$,$(I_{3720})^{-2}$,$(I_{3720})^{-1}$,$(I_{3720})^{2}$,$\log I_{3533}$,$(I_{3533})^{-2}$,$(I_{3533})^{-1}$,$(I_{3533})^{2}$,$\log I_{3405}$,$(I_{3405})^{-2}$,$(I_{3405})^{-1}$,$(I_{3405})^{2}$,$\log I_{3302}$,$(I_{3302})^{-2}$,$(I_{3302})^{-1}$,$(I_{3302})^{2}$,$\log I_{3201}$,$(I_{3201})^{-2}$,$(I_{3201})^{-1}$,$(I_{3201})^{2}$,$\log I_{3132}$,$(I_{3132})^{-2}$,$(I_{3132})^{-1}$,$(I_{3132})^{2}$,$\log I_{3001}$,...,$\frac{I_{1428}}{I_{1120}}$,$\frac{I_{1428}}{I_{1077}}$,$\frac{I_{1428}}{I_{1046}}$,$\frac{I_{1428}}{I_{881}}$,$\frac{I_{1392}}{I_{1237}}$,$\frac{I_{1392}}{I_{1180}}$,$\frac{I_{1392}}{I_{1120}}$,$\frac{I_{1392}}{I_{1077}}$,$\frac{I_{1392}}{I_{1046}}$,$\frac{I_{1392}}{I_{881}}$,$\frac{I_{1237}}{I_{1180}}$,$\frac{I_{1237}}{I_{1120}}$,$\frac{I_{1237}}{I_{1077}}$,$\frac{I_{1237}}{I_{1046}}$,$\frac{I_{1237}}{I_{881}}$,$\frac{I_{1180}}{I_{1120}}$,$\frac{I_{1180}}{I_{1077}}$,$\frac{I_{1180}}{I_{1046}}$,$\frac{I_{1180}}{I_{881}}$,$\frac{I_{1120}}{I_{1077}}$,$\frac{I_{1120}}{I_{1046}}$,$\frac{I_{1120}}{I_{881}}$,$\frac{I_{1077}}{I_{1046}}$,$\frac{I_{1077}}{I_{881}}$,$\frac{I_{1046}}{I_{881}}$
division_1_size_bulk,-2.469803,139.715275,11.820122,0.007157,-0.679671,3.893629,1.973228,0.256830,-0.063405,1.135201,1.065458,0.880902,0.000000,1.000000,1.000000,1.000000,-0.204294,1.504691,1.226658,0.664588,-0.327434,1.924890,1.387404,0.519510,-0.696649,...,0.915258,0.821316,0.806200,2.439297,1.490446,1.590381,1.099978,0.987076,0.968909,2.931602,1.067050,0.738019,0.662269,0.650080,1.966929,0.691644,0.620654,0.609231,1.843333,0.897360,0.880844,2.665147,0.981594,2.969985,3.025674
division_1_size_5,-3.264621,684.878748,26.170188,0.001460,-0.426932,2.348706,1.532549,0.425766,0.000000,1.000000,1.000000,1.000000,-0.059144,1.125569,1.060928,0.888440,-0.204907,1.506537,1.227411,0.663774,-0.349671,2.012430,1.418601,0.496912,-0.721192,...,0.880830,0.934677,1.123262,2.492599,1.450768,1.431073,1.046354,1.110320,1.334344,2.961004,0.986424,0.721242,0.765333,0.919750,2.040990,0.731168,0.775865,0.932408,2.069079,1.061132,1.275231,2.829829,1.201765,2.666802,2.219072
division_1_size_3,-2.597003,180.188987,13.423449,0.005550,-0.513285,2.791477,1.670771,0.358233,0.000000,1.000000,1.000000,1.000000,-0.064643,1.138015,1.066778,0.878723,-0.273581,1.728341,1.314664,0.578589,-0.450070,2.459949,1.568422,0.406513,-0.796935,...,0.676295,0.733057,0.863780,2.115259,1.556094,1.195628,0.776426,0.841592,0.991671,2.428442,0.768352,0.498958,0.540836,0.637282,1.560601,0.649388,0.703891,0.829414,2.031102,1.083931,1.277225,3.127719,1.178327,2.885534,2.448839


In [11]:
engineered_features_intervals_df = generate_transformed_features(features_df, features_intervals)

print(engineered_features_intervals_df.shape)
engineered_features_intervals_df.head(3)

(75, 459)


,$\log \langle I_{3700\text{–}3800} \rangle$,$(\langle I_{3700\text{–}3800} \rangle)^{-2}$,$(\langle I_{3700\text{–}3800} \rangle)^{-1}$,$(\langle I_{3700\text{–}3800} \rangle)^{2}$,$\log \langle I_{3600\text{–}3700} \rangle$,$(\langle I_{3600\text{–}3700} \rangle)^{-2}$,$(\langle I_{3600\text{–}3700} \rangle)^{-1}$,$(\langle I_{3600\text{–}3700} \rangle)^{2}$,$\log \langle I_{3500\text{–}3600} \rangle$,$(\langle I_{3500\text{–}3600} \rangle)^{-2}$,$(\langle I_{3500\text{–}3600} \rangle)^{-1}$,$(\langle I_{3500\text{–}3600} \rangle)^{2}$,$\log \langle I_{3400\text{–}3500} \rangle$,$(\langle I_{3400\text{–}3500} \rangle)^{-2}$,$(\langle I_{3400\text{–}3500} \rangle)^{-1}$,$(\langle I_{3400\text{–}3500} \rangle)^{2}$,$\log \langle I_{3300\text{–}3400} \rangle$,$(\langle I_{3300\text{–}3400} \rangle)^{-2}$,$(\langle I_{3300\text{–}3400} \rangle)^{-1}$,$(\langle I_{3300\text{–}3400} \rangle)^{2}$,$\log \langle I_{3200\text{–}3300} \rangle$,$(\langle I_{3200\text{–}3300} \rangle)^{-2}$,$(\langle I_{3200\text{–}3300} \rangle)^{-1}$,$(\langle I_{3200\text{–}3300} \rangle)^{2}$,$\log \langle I_{3100\text{–}3200} \rangle$,...,$\frac{\langle I_{1500\text{–}1600} \rangle}{\langle I_{1100\text{–}1200} \rangle}$,$\frac{\langle I_{1500\text{–}1600} \rangle}{\langle I_{1000\text{–}1100} \rangle}$,$\frac{\langle I_{1500\text{–}1600} \rangle}{\langle I_{900\text{–}1000} \rangle}$,$\frac{\langle I_{1500\text{–}1600} \rangle}{\langle I_{800\text{–}900} \rangle}$,$\frac{\langle I_{1400\text{–}1500} \rangle}{\langle I_{1300\text{–}1400} \rangle}$,$\frac{\langle I_{1400\text{–}1500} \rangle}{\langle I_{1200\text{–}1300} \rangle}$,$\frac{\langle I_{1400\text{–}1500} \rangle}{\langle I_{1100\text{–}1200} \rangle}$,$\frac{\langle I_{1400\text{–}1500} \rangle}{\langle I_{1000\text{–}1100} \rangle}$,$\frac{\langle I_{1400\text{–}1500} \rangle}{\langle I_{900\text{–}1000} \rangle}$,$\frac{\langle I_{1400\text{–}1500} \rangle}{\langle I_{800\text{–}900} \rangle}$,$\frac{\langle I_{1300\text{–}1400} \rangle}{\langle I_{1200\text{–}1300} \rangle}$,$\frac{\langle I_{1300\text{–}1400} \rangle}{\langle I_{1100\text{–}1200} \rangle}$,$\frac{\langle I_{1300\text{–}1400} \rangle}{\langle I_{1000\text{–}1100} \rangle}$,$\frac{\langle I_{1300\text{–}1400} \rangle}{\langle I_{900\text{–}1000} \rangle}$,$\frac{\langle I_{1300\text{–}1400} \rangle}{\langle I_{800\text{–}900} \rangle}$,$\frac{\langle I_{1200\text{–}1300} \rangle}{\langle I_{1100\text{–}1200} \rangle}$,$\frac{\langle I_{1200\text{–}1300} \rangle}{\langle I_{1000\text{–}1100} \rangle}$,$\frac{\langle I_{1200\text{–}1300} \rangle}{\langle I_{900\text{–}1000} \rangle}$,$\frac{\langle I_{1200\text{–}1300} \rangle}{\langle I_{800\text{–}900} \rangle}$,$\frac{\langle I_{1100\text{–}1200} \rangle}{\langle I_{1000\text{–}1100} \rangle}$,$\frac{\langle I_{1100\text{–}1200} \rangle}{\langle I_{900\text{–}1000} \rangle}$,$\frac{\langle I_{1100\text{–}1200} \rangle}{\langle I_{800\text{–}900} \rangle}$,$\frac{\langle I_{1000\text{–}1100} \rangle}{\langle I_{900\text{–}1000} \rangle}$,$\frac{\langle I_{1000\text{–}1100} \rangle}{\langle I_{800\text{–}900} \rangle}$,$\frac{\langle I_{900\text{–}1000} \rangle}{\langle I_{800\text{–}900} \rangle}$
division_1_size_bulk,-2.931331,351.658936,18.752571,0.002844,-1.578931,23.520234,4.849766,0.042517,-0.733514,4.336330,2.082386,0.230610,-0.207573,1.514592,1.230688,0.660244,-0.016212,1.032956,1.016344,0.968096,-0.108096,1.241341,1.114155,0.805581,-0.299176,...,0.802252,0.632888,1.262168,1.824879,0.753337,0.970571,0.856980,0.676062,1.348270,1.949367,1.288363,1.137579,0.897423,1.789731,2.587645,0.882965,0.696561,1.389151,2.008475,0.788889,1.573280,2.274694,1.994300,2.883416,1.445829
division_1_size_5,-3.797057,1986.467253,44.569802,0.000503,-1.268146,12.632740,3.554257,0.079159,-0.467189,2.545631,1.595503,0.392830,-0.100429,1.222451,1.105645,0.818029,-0.018644,1.037991,1.018819,0.963399,-0.127751,1.291110,1.136270,0.774527,-0.310443,...,0.769082,0.826270,1.448366,1.927351,0.772

In [12]:
all_features_df = pd.concat([features_df, engineered_features_peaks_df, engineered_features_intervals_df], axis=1)

print(all_features_df.shape)
all_features_df.head(3)

(75, 972)


,$I_{3720}$,$I_{3533}$,$I_{3405}$,$I_{3302}$,$I_{3201}$,$I_{3132}$,$I_{3001}$,$I_{2926}$,$I_{2560}$,$I_{2464}$,$I_{2345}$,$I_{2239}$,$I_{1737}$,$I_{1704}$,$I_{1651}$,$I_{1609}$,$I_{1584}$,$I_{1557}$,$I_{1526}$,$I_{1428}$,$I_{1392}$,$I_{1237}$,$I_{1180}$,$I_{1120}$,$I_{1077}$,...,$\frac{\langle I_{1500\text{–}1600} \rangle}{\langle I_{1100\text{–}1200} \rangle}$,$\frac{\langle I_{1500\text{–}1600} \rangle}{\langle I_{1000\text{–}1100} \rangle}$,$\frac{\langle I_{1500\text{–}1600} \rangle}{\langle I_{900\text{–}1000} \rangle}$,$\frac{\langle I_{1500\text{–}1600} \rangle}{\langle I_{800\text{–}900} \rangle}$,$\frac{\langle I_{1400\text{–}1500} \rangle}{\langle I_{1300\text{–}1400} \rangle}$,$\frac{\langle I_{1400\text{–}1500} \rangle}{\langle I_{1200\text{–}1300} \rangle}$,$\frac{\langle I_{1400\text{–}1500} \rangle}{\langle I_{1100\text{–}1200} \rangle}$,$\frac{\langle I_{1400\text{–}1500} \rangle}{\langle I_{1000\text{–}1100} \rangle}$,$\frac{\langle I_{1400\text{–}1500} \rangle}{\langle I_{900\text{–}1000} \rangle}$,$\frac{\langle I_{1400\text{–}1500} \rangle}{\langle I_{800\text{–}900} \rangle}$,$\frac{\langle I_{1300\text{–}1400} \rangle}{\langle I_{1200\text{–}1300} \rangle}$,$\frac{\langle I_{1300\text{–}1400} \rangle}{\langle I_{1100\text{–}1200} \rangle}$,$\frac{\langle I_{1300\text{–}1400} \rangle}{\langle I_{1000\text{–}1100} \rangle}$,$\frac{\langle I_{1300\text{–}1400} \rangle}{\langle I_{900\text{–}1000} \rangle}$,$\frac{\langle I_{1300\text{–}1400} \rangle}{\langle I_{800\text{–}900} \rangle}$,$\frac{\langle I_{1200\text{–}1300} \rangle}{\langle I_{1100\text{–}1200} \rangle}$,$\frac{\langle I_{1200\text{–}1300} \rangle}{\langle I_{1000\text{–}1100} \rangle}$,$\frac{\langle I_{1200\text{–}1300} \rangle}{\langle I_{900\text{–}1000} \rangle}$,$\frac{\langle I_{1200\text{–}1300} \rangle}{\langle I_{800\text{–}900} \rangle}$,$\frac{\langle I_{1100\text{–}1200} \rangle}{\langle I_{1000\text{–}1100} \rangle}$,$\frac{\langle I_{1100\text{–}1200} \rangle}{\langle I_{900\text{–}1000} \rangle}$,$\frac{\langle I_{1100\text{–}1200} \rangle}{\langle I_{800\text{–}900} \rangle}$,$\frac{\langle I_{1000\text{–}1100} \rangle}{\langle I_{900\text{–}1000} \rangle}$,$\frac{\langle I_{1000\text{–}1100} \rangle}{\langle I_{800\text{–}900} \rangle}$,$\frac{\langle I_{900\text{–}1000} \rangle}{\langle I_{800\text{–}900} \rangle}$
division_1_size_bulk,0.084601,0.506784,0.938564,1.000000,0.815223,0.720771,0.498252,0.422644,0.219016,0.143296,0.248711,0.118391,0.378890,0.423317,0.827933,1.0,0.761398,0.494486,0.328826,0.682168,0.819844,0.550066,0.515502,0.745328,0.830578,...,0.802252,0.632888,1.262168,1.824879,0.753337,0.970571,0.856980,0.676062,1.348270,1.949367,1.288363,1.137579,0.897423,1.789731,2.587645,0.882965,0.696561,1.389151,2.008475,0.788889,1.573280,2.274694,1.994300,2.883416,1.445829
division_1_size_5,0.038211,0.652508,1.000000,0.942571,0.814723,0.704920,0.486172,0.448983,0.201775,0.158787,0.191185,0.123498,0.371442,0.417765,0.842104,1.0,0.731738,0.455388,0.277878,0.621557,0.738359,0.508943,0.515948,0.705649,0.664996,...,0.769082,0.826270,1.448366,1.927351,0.772250,0.956124,0.805630,0.865536,1.517196,2.018944,1.238101,1.043224,1.120797,1.964643,2.614364,0.842600,0.905255,1.586819,2.111592,1.074359,1.883242,2.506043,1.752897,2.332593,1.330707
division_1_size_3,0.074497,0.598526,1.000000,0.937402,0.760651,0.637583,0.450708,0.547936,0.203027,0.137535,0.304164,0.126319,0.312673,0.408867,0.852727,1.0,0.819926,0.589477,0.391324,0.644534,0.739963,0.475526,0.618890,0.953037,0.879241,...,0.726594,0.757253,1.418718,2.162281,0.862334,1.090712,0.662785,0.690752,1.294129,1.972393,1.264837,0.768595,0.801026,1.500728,2.287273,0.607663,0.633304,1.186500,1.808355,1.042196,1.952561,2.975915,1.873506,2.855428,1.524109


## Creating features description

### `base_...`

In [13]:
base_features_peaks = features_peaks.copy()
base_features_intervals = features_intervals.copy()

In [14]:
base_features_peaks_3800_2200 = [feature for feature in base_features_peaks if 2200 <= int(feature.split('{')[1].split('}')[0]) <= 3800]
base_features_peaks_1900_800 = [feature for feature in base_features_peaks if 800 <= int(feature.split('{')[1].split('}')[0]) <= 1900]

In [15]:
base_features_intervals_3800_2200 = [feature for feature in base_features_intervals if 2200 <= int(feature.split('{')[1].split('}')[0].split('\\')[0]) <= 3800]
base_features_intervals_1900_800 = [feature for feature in base_features_intervals if 800 <= int(feature.split('{')[1].split('}')[0].split('\\')[0]) <= 1900]

### `engineered_...`

In [16]:
engineered_features_peaks = engineered_features_peaks_df.columns.tolist()
engineered_features_intervals = engineered_features_intervals_df.columns.tolist()

### `all_...`

In [17]:
all_features = all_features_df.columns.tolist()

### *summary*

In [18]:
print(
    f'(*) base_features_peaks: {len(base_features_peaks)}',
    f'(*) base_features_intervals: {len(base_features_intervals)}',
    f'(*) base_features_peaks_3800_2200: {len(base_features_peaks_3800_2200)}',
    f'(*) base_features_peaks_1900_800: {len(base_features_peaks_1900_800)}',
    f'(*) base_features_intervals_3800_2200: {len(base_features_intervals_3800_2200)}',
    f'(*) base_features_intervals_1900_800: {len(base_features_intervals_1900_800)}',
    f'(*) engineered_features_peaks: {len(engineered_features_peaks)}',
    f'(*) engineered_features_intervals: {len(engineered_features_intervals)}',
    f'(*) all_features: {len(all_features)}',
    sep='\n'
)

(*) base_features_peaks: 27
(*) base_features_intervals: 27
(*) base_features_peaks_3800_2200: 12
(*) base_features_peaks_1900_800: 15
(*) base_features_intervals_3800_2200: 16
(*) base_features_intervals_1900_800: 11
(*) engineered_features_peaks: 459
(*) engineered_features_intervals: 459
(*) all_features: 972


In [19]:
features_description_dict = {
    'base_features_peaks': base_features_peaks,
    'base_features_intervals': base_features_intervals,
    'base_features_peaks_3800_2200': base_features_peaks_3800_2200,
    'base_features_peaks_1900_800': base_features_peaks_1900_800,
    'base_features_intervals_3800_2200': base_features_intervals_3800_2200,
    'base_features_intervals_1900_800': base_features_intervals_1900_800,
    'engineered_features_peaks': engineered_features_peaks,
    'engineered_features_intervals': engineered_features_intervals,
    'all_features': all_features,
}

# Saving data

In [20]:
path_to_save = './../data/processed_data/ftir_features_long_list.csv'

all_features_df.to_csv(path_to_save, sep=';')

In [21]:
path_to_save = './../data/processed_data/features_description.json'

with open(path_to_save, 'w', encoding='utf-8') as f:
    json.dump(features_description_dict, f, ensure_ascii=False, indent=2)